In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sjoert.stellar
import pandas as pd
import os
from tqdm import tqdm
import forced_phot as fp #import flux_jy, chi2, gaussian, chi2_peak_finder
from scipy.constants import h,c,k
import json
from numpyencoder import NumpyEncoder #by Hunter M. Allen (https://pypi.org/project/numpyencoder/)

param_path = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\params.txt'


We want: \
nearestrefmag\
nearestrefmagunc

In [3]:
param_df = pd.read_csv(param_path, sep=",")
columns_old = list(param_df.keys())
columns_new = [c.strip() for c in columns_old]
columns_new[0] = 'ZTF_ID'
column_dict = dict(zip(columns_old,columns_new))
param_df.rename(columns=column_dict,inplace=True)

ztf_ids = param_df['ZTF_ID'].to_numpy()


In [43]:
datapath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data'
columns = ['sindex', 'field', 'ccdid', 'qid', 'filter', 'pid', 'infobitssci', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'zpmaginpsci', 'zpmaginpsciunc', 'zpmaginpscirms', 'clrcoeff', 'clrcoeffunc', 'ncalmatches', 'exptime', 'adpctdif1', 'adpctdif2', 'diffmaglim', 'zpdiff', 'programid', 'jd', 'rfid', 'forcediffimflux', 'forcediffimfluxunc', 'forcediffimsnr', 'forcediffimchisq', 'forcediffimfluxap', 'forcediffimfluxuncap', 'forcediffimsnrap', 'aperturecorr', 'dnearestrefsrc', 'nearestrefmag', 'nearestrefmagunc', 'nearestrefchi', 'nearestrefsharp', 'refjdstart', 'refjdend', 'procstatus']
dtypes = [(columns[x],float) for x in range(len(columns))]
dtypes[4] = ('filter',r'U8')
refmag_g_column,refmag_g_unc_column = [],[]
refmag_r_column,refmag_r_unc_column = [],[]

def get_refmags():
    for i,ztf in tqdm(enumerate(ztf_ids)):
        # if i == 5:
            # break
        year = ztf[3:5]
        ztfpath = os.path.join(datapath,year,ztf)
        batchrq = os.listdir(ztfpath)[0]

        unclean_data = pd.DataFrame(np.genfromtxt(os.path.join(ztfpath,batchrq),skip_header=53,dtype=dtypes))
        unclean_data = unclean_data[unclean_data['forcediffimfluxunc'] > 0] #these need to be removed or plotting doesn't work, usually not a lot of datapoints.

        filters = unclean_data['filter'].to_numpy()
        #for now we skip IR data so we only check ztf_g and ztf_r

        greenmask = filters == ['ZTF_g']
        redmask = filters == ['ZTF_r']

        #first for green
        if np.sum(greenmask) != 0: #if there is any g_data at all, otherwise set the refmag to 0 so we know it's missing here
            filterdata = unclean_data[greenmask]
            fields_g, ffreq_g = np.unique(filterdata['field'],return_counts=True)
            primary_field_g = fields_g[np.argmax(ffreq_g)]
            prim_field_mask_g = filterdata['field'] == primary_field_g
            # prim_field_mask = np.ones(prim_field_mask.shape).astype(bool)
            refmag = filterdata['nearestrefmag'][prim_field_mask_g].to_numpy()
            refmagunc = filterdata['nearestrefmagunc'][prim_field_mask_g].to_numpy()
                              
            nanmask = ~np.isnan(refmag)
            if np.sum(nanmask) == 0:
                refmag_green,refmag_unc_green = 0,0
            else:
                refmag_green = np.nanmean(refmag)
                refmag_unc_green = np.nanmean(refmagunc)

        else:
            refmag_green = 0
            refmag_unc_green = 0

        if np.sum(redmask) != 0: #same for red
            filterdata = unclean_data[redmask]
            fields_r, ffreq_r = np.unique(filterdata['field'],return_counts=True)
            primary_field_r = fields_r[np.argmax(ffreq_r)]
            prim_field_mask_r = filterdata['field'] == primary_field_r
            # prim_field_mask = np.ones(prim_field_mask.shape).astype(bool)
            refmag = filterdata['nearestrefmag'][prim_field_mask_r].to_numpy()
            refmagunc = filterdata['nearestrefmagunc'][prim_field_mask_r].to_numpy()
            
            nanmask = ~np.isnan(refmag)
            if np.sum(nanmask) == 0:
                refmag_red,refmag_unc_red = 0,0
            else:
                refmag_red = np.nanmean(refmag)
                refmag_unc_red = np.nanmean(refmagunc)

        else:
            refmag_red = 0
            refmag_unc_red = 0


        refmag_g_column.append(refmag_green)
        refmag_g_unc_column.append(refmag_unc_green)
        refmag_r_column.append(refmag_red)
        refmag_r_unc_column.append(refmag_unc_red)

# get_refmags()

0it [00:00, ?it/s]

13562it [10:45, 21.02it/s]


In [58]:
param_df['ZTF_ID'].shape,len(np.unique(param_df['ZTF_ID']))

((13562,), 13467)

In [48]:
refmag_g_column,refmag_g_unc_column = np.array(refmag_g_column),np.array(refmag_g_unc_column)
refmag_r_column,refmag_r_unc_column = np.array(refmag_r_column),np.array(refmag_r_unc_column)

# refflux_g = sjoert.stellar.mag2flux(refmag_g_column)
nandices_g = np.where(refmag_g_column == 0)[0]
nandices_r = np.where(refmag_r_column == 0)[0]
nan_greens = ztf_ids[nandices_g]
nan_reds = ztf_ids[nandices_r]

print(f'green: {nan_greens}, red: {nan_reds}')
print(f'green: {nan_greens.shape[0]}, red: {nan_reds.shape[0]}')
print(np.sum(np.isin(nan_greens,nan_reds)))

# for ztf in nan_greens:
#     print(ztf)
#     year = ztf[3:5]
#     ztfpath = os.path.join(datapath,year,ztf)
#     batchrq = os.listdir(ztfpath)[0]

#     unclean_data = pd.DataFrame(np.genfromtxt(os.path.join(ztfpath,batchrq),skip_header=53,dtype=dtypes))
#     unclean_data = unclean_data[unclean_data['forcediffimfluxunc'] > 0] #these need to be removed or plotting doesn't work, usually not a lot of datapoints.

#     fields, ffreq = np.unique(unclean_data['field'],return_counts=True)
#     primary_field = fields[np.argmax(ffreq)]
#     prim_field_mask = unclean_data['field'] == primary_field
#     prim_field_mask = np.ones(prim_field_mask.shape).astype(bool)
#     refmag = unclean_data['nearestrefmag'][prim_field_mask].to_numpy()
#     refmagunc = unclean_data['nearestrefmagunc'][prim_field_mask].to_numpy()
#     filters = unclean_data['filter'][prim_field_mask].to_numpy()
#     #for now we skip IR data so we only check ztf_g and ztf_r

#     greenmask = filters == ['ZTF_g']
#     redmask = filters == ['ZTF_r']
#     print(primary_field)
#     print(unclean_data[~np.isnan(refmag)].to_markdown())
    

green: ['ZTF18aashiai' 'ZTF18aazembj' 'ZTF18abvyvlr' 'ZTF18abxfiec'
 'ZTF18acsjdxo' 'ZTF19aacrmxw' 'ZTF19aamjsyl' 'ZTF19aaniqrr'
 'ZTF19aatuepb' 'ZTF19aavoivd' 'ZTF19abiawvk' 'ZTF19abiptrq'
 'ZTF19abpaskr' 'ZTF19abybstc' 'ZTF19acxoyqf' 'ZTF19adbsqgc'
 'ZTF20aaawmia' 'ZTF20aahzyep' 'ZTF20aaoqttr' 'ZTF20abgmacg'
 'ZTF20abswlts' 'ZTF20abtfosn' 'ZTF20abzfcez' 'ZTF20achvmpk'
 'ZTF20acoiehw' 'ZTF21aabgjcz' 'ZTF21aavumzc' 'ZTF21abbgkgc'
 'ZTF21abcsxah' 'ZTF21abrfxum' 'ZTF21acfjpbo' 'ZTF22aaetkzi'
 'ZTF22aaetsqd' 'ZTF22aallqgf' 'ZTF22aaomvhd' 'ZTF22abmzdej'], red: ['ZTF17aaburzk' 'ZTF17aaburzk' 'ZTF17aaburzk' 'ZTF18acbxphq'
 'ZTF18achbdoq' 'ZTF18acrvlud' 'ZTF19aaozuhj' 'ZTF20acwnmhc'
 'ZTF21aaqzsih' 'ZTF23aadrtji']
green: 36, red: 10
0


In [61]:
refmags_df = pd.DataFrame({'ZTF_ID':ztf_ids,"refmag_g":refmag_g_column,"refmag_unc_g":refmag_g_unc_column,"refmag_r":refmag_r_column,"refmag_unc_r":refmag_r_unc_column})
refmags_df.to_csv(os.path.join(datapath,'refmags.txt'),index=False,sep='\t')